In [41]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.core.prompts.prompts import SimpleInputPrompt

In [42]:
documents= SimpleDirectoryReader("data").load_data()
print(len(documents))
print(documents[0])

64
Doc ID: 790d40a5-36af-47c7-8251-a1a07c19edf0
Text: Published as a conference paper at ICLR 2023 REAC T: S
YNERGIZING REASONING AND ACTING IN LANGUAGE MODELS Shunyu Yao∗*,1,
Jeffrey Zhao2, Dian Yu2, Nan Du2, Izhak Shafran2, Karthik Narasimhan1,
Yuan Cao2 1Department of Computer Science, Princeton University
2Google Research, Brain team 1{shunyuy,karthikn}@princeton.edu
2{jeffreyzhao,dianyu,dunan,...


In [43]:
system_prompt=""" 
You are a knowledgeable and smart Q&A assistant.

Your goals:
- Use the provided context or retrieved documents to give accurate and relevant answers.
- If the context doesn’t contain the exact answer, say you don’t have enough information instead of guessing.
- Always summarize clearly and concisely.
- When possible, provide structured and easy-to-read responses (use bullet points or short paragraphs).
- Do not include unrelated or speculative content.
- If the user’s question is ambiguous, politely ask for clarification.
- Maintain a professional yet approachable tone.

Example style:
User: "What is LlamaIndex?"
Assistant: "LlamaIndex is a data framework that helps you connect external data (like files, databases, or APIs) to large language models. It simplifies loading, indexing, and querying data."

You always follow these principles for every response.
"""

# default format supportable by llama2
query_wrapper_prompt = SimpleInputPrompt(f"<|USER|>{system_prompt}<|ASSISTANT|>")

In [44]:
from dotenv import load_dotenv
load_dotenv()

import os
from typing import List, Optional
HF_TOKEN: Optional[str] = os.getenv("HUGGINGFACE_API_KEY")

### **Now we set the settings for entire system**

In [45]:
from llama_index.llms.huggingface_api import HuggingFaceInferenceAPI
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core import Settings

Settings.llm = HuggingFaceInferenceAPI(
    model_name="deepseek-ai/DeepSeek-V3.1",
    token=HF_TOKEN,
    provider="together",  # this will use the best provider available
)
Settings.embed_model = HuggingFaceEmbedding(model_name="sentence-transformers/all-MiniLM-L6-v2")
Settings.node_parser = SentenceSplitter(chunk_size=512, chunk_overlap=20)
Settings.context_window = 4096

2025-11-04 14:51:11,191 - INFO - Load pretrained SentenceTransformer: sentence-transformers/all-MiniLM-L6-v2


In [46]:
# a vector store index only needs an embed model
index = VectorStoreIndex.from_documents(
    documents, embed_model=Settings.embed_model, show_progress=True
)

Generating embeddings: 100%|██████████| 163/163 [00:00<00:00, 196.05it/s]


In [47]:
from banks import Prompt
from prompt_toolkit import prompt


query_engine=index.as_query_engine(
    Prompt=query_wrapper_prompt
)

In [48]:
result = query_engine.query("What is YOLO?")
print(result.response)

YOLO is a real-time object detection system that processes streaming video with low latency, under 25 milliseconds. It analyzes entire images at once during both training and testing, enabling it to capture contextual information and reduce errors compared to methods like Fast R-CNN. YOLO is highly generalizable, performing well even on new or unexpected inputs such as artwork. However, it may struggle with precisely locating small objects. The system uses a grid-based approach to predict bounding boxes and confidence scores for objects in an image, supporting end-to-end training and open-source availability.
